# [Cupoy 機器學習百日馬拉松-期末考](https://www.kaggle.com/c/ml100marathon-final-exam/)
* Version
  * V1: 使用 MobileNet 做 transfer learning
* Score  

| Version | Private Score |Public Score |  
|------|------|------|
|   V1  | 0.82600 | 0.83900 |


## Version: V1

In [1]:
import os, shutil

In [2]:
img_base = '/home/ryanyao/docker_mount/data/ml100marathon-final-exam'
train_path = os.path.join(img_base, 'train')
test_path = os.path.join(img_base, 'test')
sub_train_path = os.path.join(img_base, 'sub_train')
sub_valid_path = os.path.join(img_base, 'sub_valid')
sub_test_path = os.path.join(img_base, 'sub_test')

In [3]:
class_set = os.listdir(train_path)
print(class_set)

['dandelion', 'tulip', 'rose', 'sunflower', 'daisy']


In [4]:
# Sanity checks
for i in class_set:
    print('train/%s/ images:' %(i), len(os.listdir(os.path.join(train_path, i))))

train/dandelion/ images: 687
train/tulip/ images: 633
train/rose/ images: 515
train/sunflower/ images: 488
train/daisy/ images: 500


In [5]:
# copy 各50張圖片到 sub_valid/sub_test, 其餘的 copy 到 sub_train
def generate_dir():
    for i_step in (sub_train_path, sub_valid_path, sub_test_path):
        os.mkdir(i_step)
        
    for i_step in (sub_train_path, sub_valid_path, sub_test_path):
        for i_class in class_set:
            os.mkdir(os.path.join(i_step, i_class))
            fnames = os.listdir(os.path.join(train_path, i_class))
            
            # sub_train dir
            if (i_step == sub_train_path):
                sub_fnames = fnames[:-100]
            elif (i_step == sub_valid_path):
                sub_fnames = fnames[-100:-50]
            else:
                sub_fnames = fnames[-50:]
            for fname in sub_fnames:
                src = os.path.join(train_path, i_class, fname)
                det = os.path.join(i_step, i_class, fname)
                # print(src)
                # print(det)
                shutil.copyfile(src, det)

# generate_dir()

In [6]:
import numpy as np
import keras
from keras.layers.core import Dense, Flatten
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions

Using TensorFlow backend.


In [7]:
# Workaround Issue: "Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR"
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [8]:
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2)

In [9]:
batch_size = 8
sub_train_batches = ImageDataGenerator().flow_from_directory(sub_train_path, target_size=(224,224), batch_size=batch_size)
sub_valid_batches = ImageDataGenerator().flow_from_directory(sub_valid_path, target_size=(224,224), batch_size=batch_size)
sub_test_batches = ImageDataGenerator().flow_from_directory(sub_test_path, target_size=(224,224), batch_size=batch_size)

Found 2323 images belonging to 5 classes.
Found 250 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


In [10]:
# 檢視 image input size
print(sub_train_batches.image_shape)

(224, 224, 3)


In [11]:
#imports the mobilenet model and discards the last 1000 neuron layer.
base_model = MobileNet(include_top=False) 

x = base_model.output
# we add dense layers so that the model can learn more complex functions and classify for better results.
x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x) 
x = Dense(100, activation='relu')(x)
preds = Dense(5, activation='softmax')(x) # final layer with softmax activation

Instructions for updating:
Colocations handled automatically by placer.


/home/ryanyao/anaconda3/lib/python3.7/site-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [12]:
model = Model(inputs=base_model.input, outputs=preds)

In [13]:
# # Check the model architecture
# for i,layer in enumerate(model.layers):
#     print(i, layer.name, layer.trainable)

In [14]:
# 輸出整個網路結構
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [15]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_sub_train = sub_train_batches.n // sub_train_batches.batch_size
step_size_sub_valid = sub_valid_batches.n // sub_valid_batches.batch_size
history = model.fit_generator(generator=sub_train_batches, 
                    steps_per_epoch=step_size_sub_train, 
                    validation_data=sub_valid_batches,
                    validation_steps=step_size_sub_valid,
                    epochs=10,
                    verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
 - 26s - loss: 0.8430 - acc: 0.7134 - val_loss: 0.6874 - val_acc: 0.8024
Epoch 2/10
 - 20s - loss: 0.6294 - acc: 0.7871 - val_loss: 0.4418 - val_acc: 0.8430
Epoch 3/10
 - 18s - loss: 0.4940 - acc: 0.8316 - val_loss: 0.9940 - val_acc: 0.6901
Epoch 4/10
 - 18s - loss: 0.3866 - acc: 0.8657 - val_loss: 0.5033 - val_acc: 0.8182
Epoch 5/10
 - 19s - loss: 0.3785 - acc: 0.8667 - val_loss: 0.3957 - val_acc: 0.8636
Epoch 6/10
 - 19s - loss: 0.2939 - acc: 0.9001 - val_loss: 0.3866 - val_acc: 0.8926
Epoch 7/10
 - 19s - loss: 0.2937 - acc: 0.9022 - val_loss: 0.4790 - val_acc: 0.8264
Epoch 8/10
 - 18s - loss: 0.2733 - acc: 0.9056 - val_loss: 0.7482 - val_acc: 0.7934
Epoch 9/10
 - 18s - loss: 0.2392 - acc: 0.9228 - val_loss: 0.8850 - val_acc: 0.7810
Epoch 10/10
 - 18s - loss: 0.1993 - acc: 0.9293 - val_loss: 0.5835 - val_acc: 0.8554


In [16]:
step_size_sub_test = sub_test_batches.n // sub_test_batches.batch_size
model.evaluate_generator(generator=sub_test_batches, steps=step_size_sub_test, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=2)

[0.9155833564338184, 0.7620967741935484]

In [17]:
model.metrics_names

['loss', 'acc']

### 上傳 Kaggle

In [18]:
print(sub_train_batches.class_indices)


{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [19]:
from keras.preprocessing import image
import pandas as pd

In [20]:
submit = pd.DataFrame(columns=['id', 'flower_class'])
fnames = os.listdir(test_path)

for idx, fname in enumerate(fnames):
    # Loads an image into PIL format.
    img = image.load_img(os.path.join(test_path, fname), target_size=(224, 224))
    
    # Converts a PIL Image instance to a Numpy array. shape=(224, 224, 3)
    img_data = image.img_to_array(img)   
    
    # Insert a new axis that will appear at the `axis` position in the expanded array shape. shape=(1, 224, 224, 3)
    img_data = np.expand_dims(img_data, axis=0)                

    preds = model.predict(img_data)
    submit.loc[idx] = [fname.split('.')[0], preds.argmax()]

submit.to_csv('Day101_submission_v1.csv', index=False)